In [1]:
# import statements
import re
from collections import defaultdict
from math import *
import itertools
# from multiprocessing import Queue # might need this?
# import Queue   # this is Queue in 2.7, but queue in 3...not actually called in script?
# import copy
from math import factorial
import time


In [1]:
# these don't work -- why not?
from corpustools.exceptions import FuncLoadError
from .io import save_minimal_pairs
from corpustools.corpus.classes.lexicon import EnvironmentFilter

SyntaxError: invalid syntax (exceptions.py, line 318)

In [8]:
import pdb

In [2]:
def is_minpair(first, second, corpus_context, segment_pairs, environment_filter): # Q: how do I find or figure out what type of object each of these parameters should be? e.g. is segment_pairs a list? a tuple? a list of tuples? (corpus_context and environment_filter presumably are defined elsewhere...but still, it would be useful to know what their structure is)
    # according to the doc strings of minpair_fl (below): segment_pairs : list of length-2 tuples of str
    """Return True iff first/second are a minimal pair.
    Checks that all segments in those words are identical OR a valid segment pair
    (from segment_pairs) and fit the environment_filter, and that there is at least
    one difference between first and second.
    """
    first = getattr(first, corpus_context.sequence_type) # Q: how do I find out what .sequence_type does?
        # in contextmanagers.py: sequence_type is a string that is defined as "Sequence type to evaluate algorithms on (i.e., 'transcription')"
        # so this is saying get the attribute labeled in corpus_context.sequence_type ("transcription" or "vowels" or whatever) of the item named 'first'
        second = getattr(second, corpus_context.sequence_type)
    if len(first) != len(second):
        return False
    has_difference = False
    for i in range(len(first)):
        if first[i] == second[i]:
            continue
        elif (conflateable(first[i], second[i], segment_pairs) # this is defined below
            and fits_environment(first, second, i, environment_filter)): # this is defined below
            has_difference = True
            continue
        else:
            return False
    if has_difference:
        return True


In [ ]:
def conflateable(seg1, seg2, segment_pairs):
    """Return True iff seg1 and seg2 are exactly one of the segment pairs
    in segment_pairs (ignoring ordering of either).

    seg1 and seg2 will never be identical in the input.
    """
    for segment_pair in segment_pairs:
        seg_set = set(segment_pair)
        if seg1 in seg_set and seg2 in seg_set:
            return True
    return False

In [3]:
type(is_minpair)

function

In [5]:
segment_pairs = [("p", "t"), ("s", "f")] # yay, this is the structure I thought it should be!

In [9]:
seg_set = set(segment_pairs[1])

In [10]:
seg_set

{'f', 's'}

In [11]:
seg1 = "s"
seg2 = "f"

In [17]:
if seg1 in seg_set and seg2 in seg_set:
    print True

True


In [20]:
# Q: why do we need a set? does "in" not work with tuples?
# A: the set will be the UNIQUE members of the list / tuple / etc.
print(seg1 in segment_pairs[1])
print(seg2 in segment_pairs[1])

True
True


In [ ]:
# Q: ok, so "in" works with tuples not just sets -- why do we need to make the tuple into a set?

In [13]:
def fits_environment(w1, w2, index, environment_filter):
    """Return True iff for both w1 and w2 (tiers), the environment
    of its i'th element fits passes the environment_filter.
    """
    if not environment_filter:
        return True

    def ready_for_re(word, index):
        w = [str(seg) for seg in word]
        w[index] = '_'
        return ' '.join(w)

    w1 = ready_for_re(w1, index)
    w2 = ready_for_re(w2, index)
    env_re = make_environment_re(environment_filter) # this is defined below

    return (bool(re.search(env_re, w1)) and bool(re.search(env_re, w2)))

In [15]:
def ready_for_re(word, index):
        w = [str(seg) for seg in word]
        w[index] = '_'
        return ' '.join(w)

In [16]:
ready_for_re("hippopotamus",3)

'h i p _ o p o t a m u s'

In [ ]:
def make_environment_re(environment_filter): # again, need to know structure of env_filter
    if environment_filter.lhs:
        re_lhs = ' '.join(['('+('|'.join([seg for seg in position])+')') for position in environment_filter.lhs])
        re_lhs = re_lhs.replace('#', '^')
    else:
        re_lhs = ''

    if environment_filter.rhs:
        re_rhs = ' '.join(['('+('|'.join([seg for seg in position])+')') for position in environment_filter.rhs])
        re_rhs = re_rhs.replace('#', '$')
    else:
        re_rhs = ''

    if re_lhs and not re_lhs.endswith('^)'):
        re_lhs += ' '
    if re_rhs and not re_rhs.endswith('($'):
        re_rhs = ' ' + re_rhs
    return re_lhs + '_' + re_rhs

In [ ]:
# OK, here is the main minimal pair func load function
# need to know format of context manager
def minpair_fl(corpus_context, segment_pairs,
        relative_count_to_relevant_sounds = False, relative_count_to_whole_corpus = True, distinguish_homophones = False,
        environment_filter = None,
        stop_check = None, call_back = None):
    """Calculate the functional load of the contrast between two segments
    as a count of minimal pairs.

    Parameters
    ----------
    corpus_context : CorpusContext
        Context manager for a corpus
    segment_pairs : list of length-2 tuples of str
        The pairs of segments to be conflated.
    relative_count_to_relevant_sounds : bool, optional
        If True, divide the number of minimal pairs by the total count
        by the total number of words that contain either of the two segments.
        # changed the name of this from "relative_count" to "relative_count_to_relevant_sounds"
        # set its default to False above
    relative_count_to_whole_corpus : bool, optional
        If True, divide the number of minimal pairs by the total number of words 
        in the corpus (regardless of whether those words contain the target sounds).
        Defaults to True.
    distinguish_homophones : bool, optional
        If False, then you'll count sock~shock (sock=clothing) and
        sock~shock (sock=punch) as just one minimal pair; but if True,
        you'll overcount alternative spellings of the same word, e.g.
        axel~actual and axle~actual. False is the value used by Wedel et al.
    environment_filter : EnvironmentFilter
        Allows the user to restrict the neutralization process to segments in
        particular segmental contexts
    stop_check : callable, optional
        Optional function to check whether to gracefully terminate early
    call_back : callable, optional
        Optional function to supply progress information during the function

    Returns
    -------
    tuple(int or float, list)
        Tuple of: 0. if `relative_count_to_relevant_sounds`==False and 
        `relative_count_to_whole_corpus`==False, an int of the raw number of
        minimal pairs; if `relative_count_to_relevant_sounds`==True, a float of that
        count divided by the total number of words in the corpus that
        include either `s1` or `s2`; if `relative_count_to_whole_corpus`==True, a
        float of the raw count divided by the total number of words in the corpus; 
        and 1. list of minimal pairs.
    """

    if stop_check is not None and stop_check():
        return

    ## Count the number of words in the corpus (needed if relative_count_to_whole_corpus is True)
    num_words_in_corpus = len(corpus_context) # Q: not sure if this will work; modelled from the for loop below
    
    ## Filter out words that have none of the target segments
    ## (for relative_count_to_relevant_sounds as well as improving runtime)
    contain_target_segment = []
    if call_back is not None:
        call_back('Finding words with the specified segments...')
        call_back(0, len(corpus_context))
        cur = 0

    all_target_segments = list(itertools.chain.from_iterable(segment_pairs)) # creates a list of target segments from the list of tuples
    for w in corpus_context: # loops through the words in the context manager?
        if stop_check is not None and stop_check():
            return
        if call_back is not None:
            cur += 1
            if cur % 100 == 0:
                call_back(cur)
        tier = getattr(w, corpus_context.sequence_type)
        if any([s in tier for s in all_target_segments]):
                contain_target_segment.append(w)
    if stop_check is not None and stop_check():
        return

    ## Find minimal pairs
    minpairs = []
    if call_back is not None:
        call_back('Finding minimal pairs...')
        if len(contain_target_segment) >= 2:
            call_back(0,factorial(len(contain_target_segment))/(factorial(len(contain_target_segment)-2)*2))
        cur = 0
    for first, second in itertools.combinations(contain_target_segment, 2):
        if stop_check is not None and stop_check():
            return
        if call_back is not None:
            cur += 1
            if cur % 100 == 0:
                call_back(cur)
        if is_minpair(first, second, corpus_context, segment_pairs, environment_filter):
            ordered_pair = sorted([(first, getattr(first, corpus_context.sequence_type)),
                                   (second, getattr(second, corpus_context.sequence_type))],
                                   key = lambda x: x[1]) # sort by tier/transcription
            minpairs.append(tuple(ordered_pair))

    ## Generate output ## STILL NEED TO EDIT THIS
    if not distinguish_homophones:
        actual_minpairs = {}

        for pair in minpairs:
            if stop_check is not None and stop_check():
                return
            key = (pair[0][1], pair[1][1]) # Keys are tuples of transcriptions
            if key not in actual_minpairs:
                actual_minpairs[key] = (pair[0][0], pair[1][0]) # Values are words
            else:
                pair_freq = pair[0][0].frequency + pair[1][0].frequency
                existing_freq = actual_minpairs[key][0].frequency + \
                                actual_minpairs[key][1].frequency
                if pair_freq > existing_freq:
                    actual_minpairs[key] = (pair[0][0], pair[1][0])
        result = sum((x[0].frequency + x[1].frequency)/2
                    for x in actual_minpairs.values())
    else:
        result = sum((x[0][0].frequency + x[1][0].frequency)/2 for x in minpairs)

    if relative_count_to_relevant_sounds and len(contain_target_segment) > 0:
        result /= sum(x.frequency for x in contain_target_segment)
        
        # Q: do I need to put anything here between these if statements (like make it an else if)? or can I just have them back to back?
        # relative_count_to_relevant_sounds and relative_count_to_whole_corpus won't both be true
        # but both COULD be false, in which case, result should just stay as the raw count 
        # that I assume is returned above -- elif will ensure that only one runs; if will allow both to run
        
    if relative_count_to_whole_corpus:
        result = result / num_words_in_corpus
    
    return (result, minpairs)

In [22]:
import itertools

In [21]:
segment_pairs

[('p', 't'), ('s', 'f')]

In [23]:
list(itertools.chain.from_iterable(segment_pairs))

['p', 't', 's', 'f']

In [24]:
all_target_segments = list(itertools.chain.from_iterable(segment_pairs))

In [25]:
any([s in tier for s in all_target_segments])
# any(): Returns True if any element of the iterable is true.

NameError: name 'tier' is not defined

In [1]:
print("Hello!")

Hello!
